In [ ]:
from __future__ import print_function
from deepview import DeepView
import demo_utils as demo
from demo_utils import Net

import torch
import torch.optim as optim

import numpy as np
import time

import matplotlib.pyplot as plt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

## Load the Fashion MNIST data set and train a simple ConvNet

In [ ]:
# device will be detected automatically
# Set to 'cpu' or 'cuda:0' to set the device manually
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

trainloader, testset, testloader = demo.make_FashionMNIST_dataset()
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')

dim_img   = 28
dim_sq    = dim_img*dim_img
col_ch    = 1
n_classes = len(classes)

# init the model
torch.manual_seed(42)
torch_model = Net().to(device)
optimizer = optim.Adam(torch_model.parameters(), lr=0.001)


In [ ]:
# train the model on data including backdoors
# testing on clean test set

n_backd = 600 * 8
# backdoor 'bag' as 'trousers'
backd_a = 8 # attacked class
backd_t = 1 # target class

for epoch in range(1, 10 + 1):
    demo.train_backdoor(torch_model, device, trainloader, optimizer, epoch, backd_a=backd_a, backd_t=backd_t, n_backd=n_backd)
    #train(model, device, trainloader, optimizer, epoch)
    demo.test(torch_model, device, testloader)

## select a  data subset and add backdoors

In [ ]:
# prepare the test data
show_backdoors = 1

n_data  = 600 # dataset.__len__()
# pick random instances
np.random.seed(42)
rand_posis = np.random.permutation(len(testset))[0:n_data]

# check how many instances are from our attacked class
n_attackable = 0
for i in range(0, n_data):
    # load the data
    data      = testset.__getitem__(rand_posis[i])
    if data[1] == backd_a:
        n_attackable +=1 
        
n_att = 20
print('#points from target class:', n_attackable, ', #attacking points', n_att)
att_idx = np.zeros(n_att, dtype=int)

# load the data and add backdoors
#X    = torch.zeros([n_data+add_points, col_ch*dim_sq]).to(device)
X         = np.empty([n_data, col_ch, dim_img, dim_img])
labs      = np.empty([n_data], dtype=int)
pred_labs = np.empty([n_data], dtype=int)


if show_backdoors:
    print("Displaying backdoored points with backdoor label and predicted label")
    fig, axes = plt.subplots(4, round(n_att/4), figsize=(12, 8))

attacked = 0
for i in range(0, n_data):
    # load the data
    data      = testset.__getitem__(rand_posis[i])

    data_item = torch.zeros([1, col_ch, dim_img, dim_img]).to(device)
    data_item[:,:,:,:] = data[0]
    labs[i]   = data[1]
    
    # attack the first n_att images from attacked class
    if (attacked < n_att) & (labs[i].item() == backd_a):
        labs[i] = backd_t
        demo.add_backdoor(data_item[0])
        att_idx[attacked] = i
        attacked += 1
    
    output                  = torch_model(data_item)
    pred_labs[i]            = output.detach()[0].argmax().item()
    
    if (data[1] == backd_a) & (labs[i].item() == backd_t) & show_backdoors:
        if attacked-1 < round(n_att/4):
            curr_col = attacked-1
            cur_row  = 0
        elif attacked-1 < 2*round(n_att/4):
            curr_col = attacked-1 - round(n_att/4)
            cur_row  = 1
        elif attacked-1 < 3*round(n_att/4):
            curr_col = attacked-1 - 2*round(n_att/4)
            cur_row  = 2
        elif attacked-1 < 4*round(n_att/4):
            curr_col = attacked-1 - 3*round(n_att/4)
            cur_row  = 3
        
        axes[cur_row, curr_col].imshow(data_item[0,0].detach().cpu().numpy(), cmap='gray')
        axes[cur_row, curr_col].axis('off')
        axes[cur_row, curr_col].set_title(classes[labs[i].item()] + ", " + classes[output.detach()[0].argmax().item()])
    
    X[i,:,:,:] = data_item[0,:,:,:].detach().cpu().numpy()
    # first, load the data and add their index in the last dim
    #X[i,0:-1] = torch.reshape(data_item.detach(), [col_ch*dim_sq])
    #X[i,-1]   = i
    


## initialize and apply DeepView

In [ ]:
def visualization(image, point, prediction, label):
    fig, ax = plt.subplots()
    ax.imshow(image.squeeze())
    pred = classes[prediction]
    if label is None:
        ax.set_title('pred: %s' % pred)
    else:
        label = classes[label]
        ax.set_title('pred: %s - label: %s' % (pred, label))
    fig.show()

# --- Deep View Parameters ----
batch_size = 1024
max_samples = 500
data_shape = (col_ch, dim_img, dim_img)
n = 5
lam = .65
resolution = 100
cmap = 'tab10'
title = 'ConvNet - FashionMnist with backdoors'

deepview = DeepView(torch_model.predict_numpy, classes, max_samples, batch_size, data_shape, 
                    n, lam, resolution, cmap, title=title, data_viz=visualization)

In [ ]:
from deepview.evaluate import evaluate
# run deepview

umapParms = {
    "random_state": 42*42,
    "n_neighbors": 30,
    "min_dist" : 1,
    "verbose" : True
} 
deepview._init_mappers(None, None, umapParms)
#deepview.resolution = 200 # uncomment to increase resolution
# TODO: a = 400

t0 = time.time()
# create a visualization
deepview.add_samples(X, labs)

#deepview.update_mappings()

deepview.show()

print('Time to calculate visualization for %d samples: %.2f sec' % (n_data, time.time() - t0))

# calculate the quality of the projection (pi)
print('Evaluation of DeepView: %s\n' % deepview.title)
evaluate(deepview, deepview.samples, deepview.y_true)